<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#Datetime-features" data-toc-modified-id="Datetime-features-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Datetime features</a></span></li><li><span><a href="#Click-&amp;-download-stats" data-toc-modified-id="Click-&amp;-download-stats-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Click &amp; download stats</a></span></li></ul></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#One-hot-encode-categorical-features" data-toc-modified-id="One-hot-encode-categorical-features-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>One-hot encode categorical features</a></span></li></ul></li><li><span><a href="#Save-data-objects" data-toc-modified-id="Save-data-objects-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Save data objects</a></span></li></ul></div>

## Load

Packages

In [1]:
# Data tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2

# Programming tools
import os
import sys
import gc

# Notebook options
%matplotlib inline

Data

## Feature Engineering

Because of the large size of training data, my computer simply can't handle it. I am using a Postgres database to do the feature engineering tasks.

The cells with Python codes originally written for feature engineering are turned to "raw" mode, so they won't be executed.

Two tables have been previously created in Postgres. They are named `train_raw` and `test_raw`, both under schema `talkingdata`, in database `kaggle`  
Here's the list tables output from `psql` console:

Data in CSV files were imported into Postgres database with `COPY` commands. ([Details about Postgres `COPY` command](http://www.postgresqltutorial.com/import-csv-file-into-posgresql-table/))

In [6]:
from db_config import db_config
conn = psycopg2.connect(**db_config)
cur = conn.cursor()

### Datetime features

In [22]:
with open('../sql/extend_datetime_features_template.sql', 'r') as f:
    template = f.read()
print(template)

CREATE OR REPLACE VIEW {schema_name}.{out_table_name} AS
SELECT
    *,
    DATE_PART('hour', click_time) AS click_hour,
    DATE_PART('minute', click_time) AS click_minute,
    DATE_PART('second', click_time) AS click_second,
    DATE_PART('minute', click_time)::INTEGER % 15 AS click_minute_mod15,
    DATE_PART('second', click_time)::INTEGER % 5 AS click_second_mod5
FROM
    {schema_name}.{in_table_name}
;


In [24]:
for dataset in ['train', 'test']:
    query = template.format(schema_name='talkingdata', 
                            in_table_name=dataset+'_raw', 
                            out_table_name=dataset+'_timedetails')
    print(query)
    cur.execute(query)
conn.commit()

CREATE OR REPLACE VIEW talkingdata.train_timedetails AS
SELECT
    *,
    DATE_PART('hour', click_time) AS click_hour,
    DATE_PART('minute', click_time) AS click_minute,
    DATE_PART('second', click_time) AS click_second,
    DATE_PART('minute', click_time)::INTEGER % 15 AS click_minute_mod15,
    DATE_PART('second', click_time)::INTEGER % 5 AS click_second_mod5
FROM
    talkingdata.train_raw
;
CREATE OR REPLACE VIEW talkingdata.test_timedetails AS
SELECT
    *,
    DATE_PART('hour', click_time) AS click_hour,
    DATE_PART('minute', click_time) AS click_minute,
    DATE_PART('second', click_time) AS click_second,
    DATE_PART('minute', click_time)::INTEGER % 15 AS click_minute_mod15,
    DATE_PART('second', click_time)::INTEGER % 5 AS click_second_mod5
FROM
    talkingdata.test_raw
;


Unused python pandas code

### Click & download stats

In [40]:
with open('../sql/get_stats_template.sql', 'r') as f:
    template = f.read()
print(template)

DROP VIEW IF EXISTS {schema_name}.{out_table_name};

CREATE VIEW {schema_name}.{out_table_name} AS
SELECT
    {by_feature},
    COUNT(*) AS clicks_by_{by_feature},
    SUM(is_attributed) AS downloads_by_{by_feature},
    SUM(is_attributed)::FLOAT / COUNT(*) AS download_ratio_by_{by_feature} 
FROM
    {schema_name}.{in_table_name}
GROUP BY
    {by_feature}
;


In [46]:
for ft in ['ip', 'app', 'device', 'os', 'channel']:
    query = template.format(schema_name='talkingdata', 
                            in_table_name='train_raw', 
                            out_table_name='stats_by_'+ft,
                            by_feature=ft
                           )
#     print(query)
    print('Create view:', 'stats_by_'+ft)
    cur.execute(query)
conn.commit()

Create view: stats_by_ip
Create view: stats_by_app
Create view: stats_by_device
Create view: stats_by_os
Create view: stats_by_channel


## Preprocessing

In [ ]:
train.info()

In [ ]:
print(list(train.columns))

In [ ]:
cat_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_minute_mod15', 'click_second_mod5']
num_cols = ['click_hour', 'click_minute', 'click_second', 
            'clicks_by_ip', 'downloads_by_ip', 'download_ratio_by_ip', 
            'clicks_by_app', 'downloads_by_app', 'download_ratio_by_app', 
            'clicks_by_device', 'downloads_by_device', 'download_ratio_by_device', 
            'clicks_by_os', 'downloads_by_os', 'download_ratio_by_os', 
            'clicks_by_channel', 'downloads_by_channel', 'download_ratio_by_channel']
target_col = 'is_attributed'

### One-hot encode categorical features

In [ ]:
from helper.plotting import plot_df_nunique
plot_df_nunique(train, cat_cols, log_scale=True)

In [ ]:
train[cat_cols].nunique().sum()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()

In [ ]:
%%time
train_onehot = onehot_encoder.fit_transform(train[cat_cols])
test_onehot = onehot_encoder.transform(test[cat_cols])

print(train_onehot.shape)
print(test_onehot.shape)

In [ ]:
from scipy import sparse

In [ ]:
%%time
X_train = sparse.hstack((train[num_cols], train_onehot))
y_train = train[target_col].values

del train
gc.collect()

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)

In [ ]:
%%time
X_test = sparse.hstack((test[num_cols], test_onehot))

del test
gc.collect()

print('X_test shape:', X_test.shape)

## Save data objects

In [ ]:
display(type(X_train))
display(type(y_train))
display(type(X_test))

In [ ]:
%%time
sparse.save_npz('../scratch/X_train', X_train)
np.save('../scratch/y_train', y_train)
sparse.save_npz('../scratch/X_test', X_test)